# PairSpec-dLLM Colab Runner

Google Drive 上に配置した Fast-dLLM / PairSpec-dLLM プロジェクトを Colab GPU でそのまま実行する手順です。各セルを上から順番に実行してください。

## 1. Google Drive をマウント
初回は認証リンクが表示されるので、コードを入力してマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. プロジェクトパスの設定と同期
- `DRIVE_PROJECT_DIR` を Google Drive 上にコピー済みの PairSpec-dLLM ルートへ変更してください。
- Drive から `/content/PairSpec-dLLM` に同期してローカル実行用ワークツリーを用意します。

In [ ]:
import os, shutil, subprocess, textwrap

DRIVE_PROJECT_DIR = '/content/drive/MyDrive/PairSpec-dLLM'  # ★必要に応じて変更
COLAB_WORKDIR = '/content/PairSpec-dLLM'

if not os.path.isdir(DRIVE_PROJECT_DIR):
    raise FileNotFoundError(f'Google Drive 上に {DRIVE_PROJECT_DIR} が見つかりません。パスを確認してください。')


if os.path.isdir(COLAB_WORKDIR):
    shutil.rmtree(COLAB_WORKDIR)

subprocess.run(['rsync', '-a', '--exclude', '.git', DRIVE_PROJECT_DIR + '/', COLAB_WORKDIR + '/'], check=True)
os.chdir(COLAB_WORKDIR)
print('Working directory:', os.getcwd())

## 3. 依存関係のインストール
Colab ランタイムに必要パッケージを導入します。

In [ ]:
!pip install -U pip
!pip install -r requirements.txt

## 4. （任意）Hugging Face Hub へのログイン
Gated モデルを使用する場合はトークンを入力してください。

In [ ]:
from huggingface_hub import login
login()  # トークンを入力

## 5. PairSpec-dLLM テスト（LLaDA）
以下のコマンドで PairSpec パイプラインを有効化した対話ループを起動します。`draft_device` は Colab では `cuda:0` のみ利用可能です。実行後にセル下部へ入力プロンプトが表示されるので、テキストを入力してください。

In [ ]:
!python -m llada.chat --pairspec \
    --draft_model GSAI-ML/LLaDA-1.5 \
    --draft_device cuda:0 \
    --gen_length 128 --steps 128 --block_size 32 \
    --use_cache --if_cache_position \
    --accept_policy lossless --draft_depth 2 --verbose

## 6. （任意）Dream での PairSpec テスト
Dream モデルを Drive に配置している場合は以下を参考にパスだけ調整して実行してください。

In [ ]:
# 例: Dream で PairSpec を有効化する場合
!python dream/demo_multiturn_chat.py --pairspec \
    --model_path Dream-org/Dream-v0-Instruct-7B \
    --draft_model Dream-org/Dream-v0-lite \
    --draft_device cuda:0 \
    --max_new_tokens 128 --steps 128 --block_length 32 \
    --temperature 0.0 --threshold 0.9 \
    --accept_policy lossless --draft_depth 2 --verbose